In [2]:
import tensorflow as tf
from keras import applications, layers, datasets, callbacks, Model

In [3]:
(x_train_full, y_train_full), (x_test, y_test) = datasets.cifar10.load_data()
x_train, y_train, x_val, y_val = x_train_full[:45000], y_train_full[:45000], x_train_full[45000:], y_train_full[45000:]

In [ ]:
inputs = layers.Input(shape=(32, 32, 3))
x = layers.Resizing(224, 224)(inputs)
x = applications.resnet.preprocess_input(x)

base = applications.ResNet50(weights="imagenet", include_top=False, input_tensor=x)
base.trainable = False

x = layers.GlobalAveragePooling2D()(base.output)
x = layers.Dense(256, activation="relu")(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(10, activation="softmax")(x)

model = Model(inputs, outputs)

2025-12-15 10:36:59.406894: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3 Max
2025-12-15 10:36:59.406928: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 48.00 GB
2025-12-15 10:36:59.406931: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 18.72 GB
2025-12-15 10:36:59.407207: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-12-15 10:36:59.407215: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


In [5]:
callbacks = [
    callbacks.EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True),
    callbacks.ReduceLROnPlateau(monitor="val_loss", patience=3),
    callbacks.ModelCheckpoint("best_model.keras", save_best_only=True)
]

In [6]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

In [7]:
model.fit(
    x_train,
    y_train,
    validation_data=(x_val, y_val),
    epochs=999,
    callbacks=callbacks
)

Epoch 1/999


2025-12-15 10:37:21.894458: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


1407/1407 ━━━━━━━━━━━━━━━━━━━━ 102s 70ms/step - accuracy: 0.7012 - loss: 1.9420 - val_accuracy: 0.8746 - val_loss: 0.5930 - learning_rate: 0.0010
Epoch 2/999
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 97s 69ms/step - accuracy: 0.8287 - loss: 0.8349 - val_accuracy: 0.8912 - val_loss: 0.3789 - learning_rate: 0.0010
Epoch 3/999
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 97s 69ms/step - accuracy: 0.8457 - loss: 0.6031 - val_accuracy: 0.9004 - val_loss: 0.3298 - learning_rate: 0.0010
Epoch 4/999
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 98s 70ms/step - accuracy: 0.8521 - loss: 0.5519 - val_accuracy: 0.8936 - val_loss: 0.3837 - learning_rate: 0.0010
Epoch 5/999
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 97s 69ms/step - accuracy: 0.8568 - loss: 0.5716 - val_accuracy: 0.8820 - val_loss: 0.4176 - learning_rate: 0.0010
Epoch 6/999
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 97s 69ms/step - accuracy: 0.8574 - loss: 0.5662 - val_accuracy: 0.8584 - val_loss: 0.6097 - learning_rate: 0.0010
Epoch 7/999
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 97s 69ms/step - accuracy: 

In [8]:
loss, accuracy = model.evaluate(x_test, y_test)


print("Test accuracy: ", accuracy)
print("Test loss: ", loss)

313/313 ━━━━━━━━━━━━━━━━━━━━ 19s 60ms/step - accuracy: 0.9103 - loss: 0.2575
Test accuracy:  0.9099000096321106
Test loss:  0.2623491585254669
